# Error Mitigation

This notebook contains theoretical background and examples for using our Error Mitigation module with Qiskit. 
We recommend to first read our [Quantum Detector Tomography Tutorial](QDT_Tutorial.ipynb).

## Theoretical background
Here we describe main theoretical concepts related to mitigation procedure. This description is based on Ref. [0].

### Classical noise model

Let us denote by $\mathbf{M}$ a POVM describing noisy detector and $\mathbf{P}$ denote ideal measurement. In classical
noise model, we assume that the relation between the two is given by stochastic, invertible map $\Lambda$:

$$ \mathbf{M} = \Lambda \mathbf{P}.$$

Let us denote by $\mathbf{p}_{exp}$ vector of probabilities obtained on a noisy detector $\mathbf{M}$ measuring any
quantum state, and by $\mathbf{p_{ideal}}$ the analogous vector for ideal detector (for the same quantum state). From
linearity of the Born's rule, it follows that two vectors are related by the same stochastic map as POVMs:

$$ \mathbf{p_{exp}} = \Lambda \mathbf{p_{ideal}}. $$

Recall that we assumed that $\Lambda$ is invertible. Hence by multiplying last equation by $\Lambda^{-1}$ from both
sides, we obtain

$$ \Lambda^{-1} \mathbf{p_{exp}} = \mathbf{p_{ideal}}. $$

Effectively, by this kind of post-processing, we obtain statistics which we would have obtained on the perfect detector
devices.

### Deviations from noise model

There are several ways in which real noise can deviate from our model. Here we will show the way how to handle them.

#### Effects  on mitigation
Above we assumed a very specific noise model. In practice it is likely that it will not be fulfilled exactly. In such a
scenario, we may perform the following decomposition of POVM $M$ describing our device:
$$ \mathbf{M} = \Lambda \mathbf{P} + \mathbf{\Delta}\ ,$$
where $\Delta$ denotes a "coherent" part of the noise and $\Lambda$, as previously, is some stochastic, invertible map. 

In such a case, we can relate probability vector obtained on the noisy detector to ideal one in manner similar as before

$$ \mathbf{p_{exp}} = \Lambda \mathbf{p_{ideal}} + \mathbf{d}\ , $$
where $\mathbf{d}$ denotes a disturbance of probability vector due to existence of coherent part of the noise.
If we now multiply the above expression by inverse of the noise matrix, we obtain

$$ \Lambda^{-1} \mathbf{p_{exp}} = \mathbf{p_{ideal}} + \Lambda^{-1} \mathbf{d}, \  $$


This clearly does not leave us with ideal statistics $\mathbf{p_{ideal}}$, but consists of additional deviation term
$\Lambda^{-1} \mathbf{d}$ which appears due to non-classicity of noise. Fortunately, this does not preclude our
mitigation from working! The correction clearly will not be perfect in this case, however if deviation is sufficiently
small, it still might be better then performing no correction at all.

We can upper-bound the TV distance of corrected probability distribution from the ideal one by expression
$$
D_{TV}\left( \Lambda^{-1} \mathbf{p_{exp}},\mathbf{p_{ideal}} \right) \leq ||\Lambda^{-1}||_{1 \to 1}
D_{op}(\mathbf{M}, \Lambda \mathbf{P}) \,
$$
where the $1\rightarrow 1$ norm is defined as
$$
||A||_{1\rightarrow1} = \sup_{||v||_1} ||Av||_1 \ .
$$

It is easy to see that if the detector $\mathbf{M}$ is related to $\mathbf{P}$ via only classical noise $\Lambda$, then
the above expression yields $0$, hence the mitigation works perfectly. In the presence of coherent measurement noise,
term $D_{op}(\mathbf{M}, \Lambda \mathbf{P})$ may be regarded as a measure of non-classicity of the noise.

#### How to choose decomposition?

In general, we can always perform decomposition into "classical" and "coherent" part in infinitely many ways. However,
for the ideal detector $P$ modeled as projective measurement in computational basis, there exists a perfectly natural
ansatz. Namely, we propose to consider diagonal parts of POVM's $\mathbf{M}$ elements to describe classical part of the
noise. As a justification for such choice, note that elements of $\mathbf{P}$ for such ideal detector model are
diagonal, and the stochastic map would preserve the diagonality.

Hence, after obtaining description of POVM $\mathbf{M}$ from detector tomography, reconstruction of $\Lambda$ can be
achieved by taking only diagonal parts of POVM's elements.

### Finite-size statistics
#### Statistical noise
So far we have assumed that we have access to actual (noisy) probability distribution $\mathbf{p_{exp}}$. However, in
real life what we can get is only an estimator of this distribution

$$ \mathbf{p_{exp}} \xrightarrow[\text{}]{\text{finite-size sampling}} \mathbf{p}_{\mathbf{exp}}^{\text{est}}\ ,$$ 
which simply consists of experimental frequencies of occurrences of particular outcomes.

As a result, we will have some statistical noise in our estimator, which we can write as
$$
\mathbf{p}_{\mathbf{exp}}^{\text{est}}  = \mathbf{p}_{\mathbf{exp}} + \mathbf{s} \ ,
$$
where $\mathbf{s}$ is some vector of statistical deviations. 

Let us assume that we perform $n$-outcome measurement and gather $k$ samples (shots). Then the magnitude of those
statistical deviations can be upper bound by
$$
D_{TV}\left(\mathbf{p}_{\mathbf{exp}}^{\text{est}} ,
\mathbf{p}_{\mathbf{exp}} \right) \leq \sqrt{\frac{\log{\left(2^n-2\right)}-\log{\text{Pr}_{wrong}}}{2k}}
\equiv \epsilon \ .
$$
Bounds on statistical deviations are usually probabilistic, and in the above equation we choose parameter
$\text{Pr}_{wrong}$ such that
$$
1-\text{Pr}_{wrong} = \text{Pr}\left(D_{TV}\left(\mathbf{p}_{\mathbf{exp}}^{\text{est}} ,
\mathbf{p}_{\mathbf{exp}} \right) \leq \epsilon\right) \ .
$$

#### Effects on mitigation
Now we want to determine how far (in TV distance) is our corrected estimated statistics vector
$\Lambda^{-1}\mathbf{p}_{\mathbf{exp}}^{\text{est}}$ from ideal statistics $\mathbf{p_{ideal}}$.
This distance can be upper bounded by

$$ D_{TV}\left(\Lambda^{-1} \mathbf{p}_{\mathbf{exp}}^{\text{est}}, \mathbf{p_{ideal}}\right) \leq
\underbrace{||\Lambda^{-1}||_{1 \to 1} D_{op}(\mathbf{M}, \Lambda \mathbf{P})}_{\text{coherent noise}} +
\underbrace{||\Lambda^{-1}||_{1 \to 1} \epsilon}_{\text{statistical noise}} =: \delta. $$

This sum consists of two parts. 
First quantifies propagation of coherent errors under our correction and is the same as in previous subsection.
The second term quantifies propagation of statistical noise. For more details see Ref. [0].

### Quasiprobability vectors
In the discussion above we have ignored a very important feature of correction matrix $\Lambda^{-1}$. Namely, since it is an inverse of stochastic matrix, its columns are in fact quasiprobability vectors - it means that they still sum up to 1, but can be negative. Matrices which such columns in general do not preserve probability vectors - hence left-multiplication of estimated frequencies by $\Lambda^{-1}$ may sometimes yield a quasiprobability vector! 

Possible way of dealing with this is to simply find a closest (in some norm) probability vector. In other words, if $\Lambda^{-1} \mathbf{p}_{\mathbf{exp}}^{\text{est}}$  turns out to be quasiprobability vector, we solve
$$
\min_{\forall_i p_i\geq 0,\ \sum_i p_i =1} ||\Lambda^{-1} \mathbf{p}_{\mathbf{exp}}^{\text{est}} - \mathbf{p}||_2 \ ,
$$
where we minimize Euclidean distance, not TV distance, because it is easy to solve. 

Let us denote by $\mathbf{p_{final}}$ the solution of the above optimization problem. In such a case, we can upper bound possible distance from ideal distribution by
$$
D_{TV}\left(\mathbf{p_{final}}, \mathbf{p_{ideal}}\right) \leq  \underbrace{\delta}_{\text{coherent and statistical noise}} + \underbrace{\alpha}_{\text{"unphysicality noise"}} \ ,
$$
where by $\alpha$ we have denoted
$$
\alpha := D_{TV}\left(\mathbf{p_{final}}, \Lambda^{-1} \mathbf{p}_{\mathbf{exp}}^{\text{est}}\right) \ ,
$$
hence a distance between initially estimated quasiprobability vector and the solution of optimization problem.


### When is mitigation successful?

We have discussed two sources of errors that can worsen results of our mitigation procedure. 
Since in real-life experiments we do not have access to the ideal probability distribution, it is  impossible to know whether the mitigation was successful. To deal with this, we use the following heuristic rule to assess whether mitigation was successful 

$$ \delta + \alpha < D_{op}(\mathbf{M}, \mathbf{P}) + \epsilon\ ,$$

where $\delta$ coherent and statistical errors in mitigation,0 $\alpha$ describes the distance described in previous subsection, and $\epsilon$ is statistical error.
In words - if a possible error introduced by mitigation is lower than the upper bound on error appearing in a detector without mitigation, we consider mitigation successful.

Here we back it up by the following arguments. First, we know that at least for some set of quantum states the above is true - namely, the states in the neighborhood of the state that saturates upper bound on the RHS of the above inequality (i.e., states which are "worst case scenarios" for noisy detector, recall definition of operational distance from [QDT tutorial](https://github.com/fbm2718/QREM/blob/master/QDT_Tutorial.ipynb)). 

Secondly, In Appendix B of Ref. [0] are results of numerical simulations for single and two-qubit (uncorrelated) detectors which confirm that in the case when the above inequality is fulfilled, the majority of noisy probability vectors (generated by random quantum states and noisy detector) indeed come closer to the ideal distribution than if one does not apply mitigation. 

Finally, in experimental section of Ref. [0] there is empirical confirmation of the procedure to work for the number of benchmark procedures (note that if we know ideal probability distribution, we clearly can assess whether mitigation has been successful - we also do such tests in practical part of this tutorial).




## Mitigating the errors using our module

In this section we will describe how to use our mitigation module. We will begin with showing general approach as to
how to prepare the mitigator object and then, on several examples, we will show how to use it
(on single and multi-qubit circuit results).

### Performing Quantum Detector Tomography
Our error mitigation approach is based on the knowledge about the noise in the device's detector. Such knowledge can be
obtained in procedure known as Quantum Detector Tomography (QDT). To perform QDT, one can follow the steps from our
[QDT tutorial](https://github.com/fbm2718/QREM/blob/master/QDT_Tutorial.ipynb).

In [1]:
import povmtools
import ancillary_functions as anf
import numpy as np

from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer import noise

from quantum_tomography_qiskit import detector_tomography_circuits
from DetectorTomographyFitter import DetectorTomographyFitter
from QDTErrorMitigator import QDTErrorMitigator

# Choose qubit indices
QDT_qubit_index = [3]

# Select probe kets
QDT_probe_kets = povmtools.pauli_probe_eigenkets

# Generate circuits
QDT_circuits = detector_tomography_circuits(QDT_qubit_index, QDT_probe_kets)

# Get QDT circuits results
backend = Aer.get_backend('qasm_simulator')  #  Get backed
shots_number = 2000  # Define number of measurement repetitions
QDT_job = execute(QDT_circuits, backend=backend, shots=shots_number)
results = QDT_job.result()

# Get ml_povm_estimator using DTF and results
DTF = DetectorTomographyFitter()
ml_povm_estimator = DTF.get_maximum_likelihood_povm_estimator([results], QDT_probe_kets)

## Preparing mitigation

Now that we have the estimator of POVM, we can create QDTErrorMitigator object and prepare it.

In [2]:
# Creation and preparation of QDTErrorMitigator
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

With prepared mitigator object we gain access to several useful functionalities. For example, we can:
* Correct results of qiskit job by using apply_correction_to_qiskit_job(Results) method.
* Access error and correction matrices obtained from POVM given during preparation.

In order to properly analyze the results of correction procedure, one have to be aware that in some cases raw
application of $\Lambda^{-1}$ to the results may yield quasi-probability (instead of probability) vectors. In such
scenario our code finds closest probability vectors and returns them instead. Distances from raw quasi-probabilities to
returned probabilities, can be accessed via distances_from_closest_probability_vector member of mitigator object.

### Error bounds calculation

With access to POVM and the error and correction matrices, we are able to calculate bounds on several errors. In
particular, using povmtools module, we can calculate:

* noisy detector errors - using povmtools.operational_distance function (denoted as $D_{op}\left(\mathbf{M},
\mathbf{P}\right)$ in theoretical background)
* statistical error bound - using get_statistical_error_bound method (denoted as $\epsilon$ in theoretical background),
* coherent error bound - using gt_coherent_error_bound method (denoted as $D_{op}\left(\mathbf{M},\Lambda
\mathbf{P}\right)$ in theoretical background),
* correction error bound - using get_correction_error_bound_from_data or get_correction_error_bound_from_parameters
 method (denoted as $\delta+\alpha$ in theoretical background).

And later after applying mitigation to particular results we can also calculate
* "unphysicality error" - using distances_from_closest_probability_vector method (denoted as $\alpha$ in theoretical
 background)

## Single qubit error mitigation

In this section two examples of mitigation for single qubit will be shown. In order to show, that our mitigation scheme
is efficient, we first need to create a noisy backend simulator. We start with required imports.

In [3]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
import pandas as pd
from pprint import pprint

import povmtools
from DetectorTomographyFitter import DetectorTomographyFitter
from quantum_tomography_qiskit import detector_tomography_circuits
from QDTErrorMitigator import QDTErrorMitigator

We use standard qiskit methods to create noisy backend simulator. For this tutorial we will create simulator of IBM's
Burlington device which is usually quite noisy. Here the assumed model is classical and uncorrelated, hence all errors
in mitigation will be due to statistical noise.

In [4]:
#  What we want to have first is working noisy backend simulation. In order to do that, we use qiskit noise model
# and download properties of selected backend.

# Build noise model from backend properties.
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_burlington')
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend, why not.
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model.
basis_gates = noise_model.basis_gates

# Finally, get the simulator backend.
simulator_backend = Aer.get_backend('qasm_simulator')

With backend created, what we need to do next is to reconstruct the POVM that corresponds to detector. To do so,
following our QDT tutorial, we use DetectorTomographyFitter object from our module. First, we need to create calibration
circuits and implement them.

In [5]:
# What I want to do now, is prepare POVM for simulated backend. According to our other notebook, I prepare
# circuits first.

qdt_qubit_index = [0]
qdt_probe_kets = povmtools.pauli_probe_eigenkets
qdt_calibration_circuits = detector_tomography_circuits(qdt_qubit_index, qdt_probe_kets)

# I then execute them on backend prepared earlier.
shots_number = 8192

# Perform a noisy simulation
result = execute(qdt_calibration_circuits, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

# Print counts.
for i in range(len(result.results)):
    counts_now = result.get_counts(i)
    frequencies_now = povmtools.counts_dict_to_frequencies_vector(counts_now)
    print(np.round(frequencies_now,3))

[0.935 0.065]
[0.031 0.969]
[0.487 0.513]
[0.48 0.52]
[0.48 0.52]
[0.479 0.521]


We note that it is highly preferable to use as high number of shots as possible to minimize statistical errors.  

Maximum likelihood POVM calculation is as easy as calling single method now.

In [6]:
# With circuits results we can now use our Detector Tomography Fitter to obtain maximum likelihood POVM estimator.
dtf = DetectorTomographyFitter()
ml_povm_estimator = dtf.get_maximum_likelihood_povm_estimator([result], qdt_probe_kets)

for m_i in ml_povm_estimator:
    nice_looking_m_i = pd.DataFrame(m_i)
    print(nice_looking_m_i)

                    0                   1
0  0.934164-0.000000j  0.003357-0.000550j
1  0.003357+0.000550j  0.030750+0.000000j
                    0                   1
0  0.065836+0.000000j -0.003357+0.000550j
1 -0.003357-0.000550j  0.969250+0.000000j


With POVM calculated, we can now create and prepare mitigator object.

In [7]:
# We can use obtained POVM to correct to prepare mitigation object.
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

After those preparations, we can now check how our error mitigation approach works. Let's consider two simple scenarios.

### X gate circuit

In this case, we will first create a one qubit circuit (initially in |0><0| state) and we will apply X (not) operation
to it. In ideal scenario we would expect only counts in state '1'. We begin with circuit creation.

In [8]:
# In order to check how efficient our mitigator is, we need to obtain some noisy data first.
# We will start with preparing simple experiment.

qr = QuantumRegister(1, 'qreg')
cr = ClassicalRegister(1, 'creg')
qc = QuantumCircuit(qr, cr)

qc.x(qr[0])
qc.measure(qr, cr);

Now we execute this circuit on our simulator and check the results.

In [9]:
result = execute(qc, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

for i in range(len(result.results)):
    counts_now = result.get_counts(i)
    frequencies_now = povmtools.counts_dict_to_frequencies_vector(counts_now)
    print(np.round(frequencies_now,3))

[0.027 0.973]


As we can see, noisy simulator returned some errors ('0' counts), as expected. Let's try to correct these results using
our mitigator.

In [10]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(np.round(corrected_results[0],3))

good format
[[0.]
 [1.]]


### H gate circuit

Now we will use H (Hadamard) gate. Everything else will be exactly like in the first scenario. We begin with creating
and executing a circuit and then printing raw results of the job. We expect to obtain equal number of '0' and '1'
counts.

In [11]:
# Let's try another example.
qr2 = QuantumRegister(1, 'qreg2')
cr2 = ClassicalRegister(1, 'creg2')
qc2 = QuantumCircuit(qr2, cr2)

qc2.h(qr2[0])
qc2.measure(qr2, cr2)

result = execute(qc2, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

for i in range(len(result.results)):
    counts_now = result.get_counts(i)
    frequencies_now = povmtools.counts_dict_to_frequencies_vector(counts_now)
    print(np.round(frequencies_now,3))

[0.483 0.517]


We see that again have distribution which is not perfect. Let's apply error mitigation.

In [12]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(np.round(corrected_results[0],3))

good format
[[0.501]
 [0.499]]


Once again, the results are better!

### Errors in mitigation
In the above benchmark circuits it was straightforward to assess the effectiveness of mitigation - since we knew the
theoretical distributions, we could simply compare the corrected statistics to this distribution.

In realistic scenarios, however, if one does not know what is the underlying ideal distribution, it is impossible to
assess the success of mitigation. However, following procedure described in [0], we can heuristically asses if it is
likely that mitigation succeeded (see theoretical background of this tutorial).

We start with calculating statistical error bound. We start with setting mistake probability (in theoretical background
denoted as $\text{Pr}_{\text{wrong}}$).

In [13]:
statistical_error_mistake_probability = 0.01

Now we need counts formatted as list or array.

In [14]:
# We need counts as array. The outcomes are 0 or 1, so we can create them in straightforward way.
counts_dict = result.get_counts()
counts_list = [counts_dict['0'], counts_dict['1']]

Now we can use povmtools method to calculate statistical error bound.

In [15]:
# We calculate statistical error bound. Let's call it epsilon.
epsilon = povmtools.get_statistical_error_bound(counts_list, statistical_error_mistake_probability)
print('statistical errors:',epsilon)

statistical errors: 0.017982870414073163


We now need bound for possible errors in our mitigation. Luckily, we have all the arguments necessary for its
calculation using prepared povmtools method.

In [16]:
#get correction matrix
correction_matrix = mitigator.correction_matrix


#do not forget about possible term from unphysicality. 
alpha = mitigator.distances_from_closest_probability_vector[0]
print('unphysicality error:',alpha)



# We now need the correction error bound. 
mitigation_error = povmtools.get_correction_error_bound_from_data_and_statistical_error(ml_povm_estimator,
                                                                             correction_matrix, 
                                                                             epsilon,
                                                                             alpha)


print('total mitigation error:',mitigation_error)

unphysicality error: 0
total mitigation error: 0.0245008574271664


Note that since we took a classical noise model from IBM's backend properties, the above error arises solely due to
statistical fluctuations and is underestimated since it does not account for coherent errors.

Last thing required for determining mitigation success is operational distance between perfect measurement and our
ml_povm_estimator. We calculate it using povmtools module function.

In [17]:
# We also need operational distance, between perfect detector and our POVM. We calculate it.
perfect_measurement = povmtools.computational_projectors(d=2) # For one qubit!
operational_distance = povmtools.operational_distance_POVMs(ml_povm_estimator, perfect_measurement)
print('errors without mitigation:',operational_distance)

errors without mitigation: 0.06595563041643235


And finally, we can check if our criterion for assesing success of the mitigation works in this case.

In [19]:
# Now we can check if mitigation can be considered successful
print(f'Mitigation successful: {mitigation_error < operational_distance + epsilon}')

Mitigation successful: True


## Multi Qubit error mitigation scenario

Our method can easily be expanded for multiple qubits. It's done analogically to the single qubit experiments. We start
with preparing noisy backend. We can use the same code as in one qubit scenario.

In [20]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel

import povmtools
from DetectorTomographyFitter import DetectorTomographyFitter
from quantum_tomography_qiskit import detector_tomography_circuits
from QDTErrorMitigator import QDTErrorMitigator

#  What I want to have first is working noisy backend simulation. In order to do that, I will use qiskit noise model
# and download properties of selected backend.

# Build noise model from backend properties.
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_burlington')
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend, why not.
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model.
basis_gates = noise_model.basis_gates

# Finally, get the simulator backend.
simulator_backend = Aer.get_backend('qasm_simulator')

ibmqfactory.load_account:WARNING:2020-04-18 00:01:02,644: Credentials are already in use. The existing account in the session will be replaced.


Now the POVM. This time we need higher dimensional one, as we are considering multi qubit scenario. The code, however,
doesn't change much. The only thing we need to do is to modify qdt_qubit_indices list.

In [21]:
# What we want to do now, is prepare POVM for simulated backend. According to our other notebook, we prepare
# circuits first.

qdt_qubit_indices = [0, 1]
qdt_probe_kets = povmtools.pauli_probe_eigenkets
qdt_calibration_circuits = detector_tomography_circuits(qdt_qubit_indices, qdt_probe_kets)

# We then execute them on backend prepared earlier.
shots_number = 8192

# Perform a noisy simulation
result = execute(qdt_calibration_circuits, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number) \
    .result()

# Print counts.
for i in range(len(result.results)):
    counts_now = result.get_counts(i)
    frequencies_now = povmtools.counts_dict_to_frequencies_vector(counts_now)
    print(np.round(frequencies_now,3))

[0.88  0.062 0.055 0.003]
[0.068 0.004 0.867 0.061]
[0.474 0.032 0.462 0.032]
[0.479 0.034 0.459 0.028]
[0.474 0.031 0.464 0.031]
[0.475 0.033 0.464 0.028]
[0.027 0.913 0.002 0.058]
[0.002 0.074 0.03  0.893]
[0.019 0.499 0.015 0.467]
[0.016 0.493 0.015 0.475]
[0.016 0.499 0.015 0.471]
[0.013 0.494 0.014 0.479]
[0.448 0.496 0.026 0.03 ]
[0.039 0.037 0.445 0.48 ]
[0.245 0.257 0.24  0.258]
[0.251 0.259 0.237 0.252]
[0.24 0.27 0.24 0.25]
[0.249 0.267 0.233 0.25 ]
[0.461 0.481 0.027 0.031]
[0.035 0.045 0.442 0.478]
[0.244 0.266 0.228 0.262]
[0.255 0.258 0.234 0.254]
[0.25  0.257 0.248 0.244]
[0.247 0.265 0.239 0.249]
[0.46  0.481 0.03  0.03 ]
[0.038 0.044 0.438 0.48 ]
[0.244 0.258 0.239 0.26 ]
[0.25  0.261 0.234 0.255]
[0.248 0.262 0.24  0.251]
[0.252 0.262 0.238 0.248]
[0.447 0.497 0.026 0.03 ]
[0.037 0.036 0.453 0.474]
[0.256 0.258 0.243 0.243]
[0.249 0.26  0.237 0.254]
[0.253 0.261 0.233 0.253]
[0.252 0.26  0.236 0.252]


Now, we can calculate the POVM estimator.

In [22]:
# With circuits results we can now use our Detector Tomography Fitter to obtain maximum likelihood POVM estimator.
dtf = DetectorTomographyFitter()
ml_povm_estimator = dtf.get_maximum_likelihood_povm_estimator([result], qdt_probe_kets)

for m_i in ml_povm_estimator:
    nice_looking_m_i = pd.DataFrame(m_i)
    print(nice_looking_m_i,'\n')

                    0                   1                   2  \
0  0.883276-0.000000j -0.003504+0.002544j -0.005574-0.001132j   
1 -0.003504-0.002544j  0.071331-0.000000j -0.000234+0.000364j   
2 -0.005574+0.001132j -0.000234-0.000364j  0.029219-0.000000j   
3  0.002282-0.006682j  0.002018-0.000776j  0.000877+0.001242j   

                    3  
0  0.002282+0.006682j  
1  0.002018+0.000776j  
2  0.000877-0.001242j  
3  0.002402+0.000000j   

                    0                   1                   2  \
0  0.053174+0.000000j  0.002040-0.002328j -0.000638-0.002447j   
1  0.002040+0.002328j  0.866528+0.000000j  0.003787-0.000379j   
2 -0.000638+0.002447j  0.003787+0.000379j  0.001512+0.000000j   
3  0.000771-0.001048j  0.001273-0.005030j  0.000062+0.000189j   

                    3  
0  0.000771+0.001048j  
1  0.001273+0.005030j  
2  0.000062-0.000189j  
3  0.028618+0.000000j   

                    0                   1                   2  \
0  0.060649-0.000000j -0.000770+0.00116

And with POVM estimator, we can prepare mitigator object.

In [23]:
# We can use obtained POVM to correct to prepare mitigation object.
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

Again, let's check how the mitigation works on several circuits.

### Hadamards circuit

We begin with circuit applying Hadamard gates on both qubits. Let's prepare such circuit.

In [24]:
# In order to check how efficient our mitigator is, we need to obtain some noisy data first.
# We will start with preparing simple experiment.

qr = QuantumRegister(2, 'qreg')
cr = ClassicalRegister(2, 'creg')
qc = QuantumCircuit(qr, cr)

qc.h(qr[0])
qc.h(qr[1])
qc.measure(qr, cr);

We then execute this circuit and print the results.

In [25]:
result = execute(qc, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number) \
    .result()

for i in range(len(result.results)):
    counts_now = result.get_counts(i)
    frequencies_now = povmtools.counts_dict_to_frequencies_vector(counts_now)
    print(frequencies_now)

[0.2418212890625, 0.2554931640625, 0.244140625, 0.258544921875]


Now we can correct them.

In [26]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(corrected_results[0])

good format
[[0.24427051]
 [0.24139189]
 [0.2578647 ]
 [0.2564729 ]]


One can clearly see that the results are better. 

Now let's it try with some multi-qubit gates.

### CNOT Circuit

This time we will use a CNOT gate. First we will prepare a qubit in state |1> and then use it as a control to negate the
other qubit. We expect all counts to be in state '11'. The circuit can be codded as below.

In [27]:
# Let's try another example.
qr2 = QuantumRegister(2, 'qreg2')
cr2 = ClassicalRegister(2, 'creg2')
qc2 = QuantumCircuit(qr2, cr2)

qc2.x(qr2[0])
qc2.cx(qr2[0], qr2[1])
qc2.measure(qr2, cr2)

Executing this circuits yields following results:

In [28]:
result = execute(qc2, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number) \
    .result()

for i in range(len(result.results)):
    counts_now = result.get_counts(i)
    frequencies_now = povmtools.counts_dict_to_frequencies_vector(counts_now)
    print(frequencies_now)

[0.00439453125, 0.0701904296875, 0.0308837890625, 0.89453125]


As expected, we can see some erroneous results. Let's try to mitigate it.

In [29]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(np.round(corrected_results[0],3))

good format
[[0.]
 [0.]
 [0.]
 [1.]]


Easily enough, the results are clearly better. Let's now combine Hadamard and CNOT gate.

### Hadamard + CNOT circuit

In this scenario, we will first prepare first qubit in superposition of states |0> and |1>. We will then, again, use it
as a control qubit to negate the second one. We will expect equal number of counts in '00' and '11' states. The circuit
can be prepared like below.

In [30]:
# Final example!
qr3 = QuantumRegister(2, 'qreg2')
cr3 = ClassicalRegister(2, 'creg2')
qc3 = QuantumCircuit(qr3, cr3)

qc3.h(qr3[0])
qc3.cx(qr3[0], qr2[1])
qc3.measure(qr3, cr3)

We now execute it and print the results.

In [31]:
result = execute(qc3, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number) \
    .result()

for i in range(len(result.results)):
    counts_now = result.get_counts(i)
    frequencies_now = povmtools.counts_dict_to_frequencies_vector(counts_now)
    print(frequencies_now)

[0.44580078125, 0.0645751953125, 0.0413818359375, 0.4482421875]


There's some room for improvement. Let's apply the mitigation.

In [32]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(np.round(corrected_results[0],3))

good format
[[0.501]
 [0.   ]
 [0.   ]
 [0.499]]


Once again, the results are clearly better.

This time it was possible, to determine if the mitigation worked without using mitigation success check. If, however,
one would like to perform it, then the only difference between multi and single qubit scenario code, would be
perfect_measurement definition (adequate to the dimension of the problem). Mitigation success check won't be, therefore,
shown for multi qubit case.

As a final remark one should be aware, that the results of the mitigation and its success in particular, strongly
depends on quantum detector tomography method used during POVM calculation. In our work maximum likelihood method was
used, but there are several different methods, that could possibly yield different results. For more information see
[0]. We can, as in single-qubit example, check if our criterion for assessing mitigation success works in this case as
well.

In [33]:
# We need counts as array. The outcomes are 0 or 1, so we can create them in straightforward way.
counts_dict = result.get_counts()
counts_list = np.array(povmtools.counts_dict_to_frequencies_vector(counts_dict))*shots_number


# We calculate statistical error bound. 
epsilon = povmtools.get_statistical_error_bound(counts_list, statistical_error_mistake_probability)
print('statistical errors:',epsilon)



#get correction matrix
correction_matrix = mitigator.correction_matrix


#do not forget about possible term from unphysicality. 
alpha = mitigator.distances_from_closest_probability_vector[0]
print('unphysicality error:',alpha)



# We now need the correction error bound.
mitigation_error = povmtools.get_correction_error_bound_from_data_and_statistical_error(ml_povm_estimator,
                                                                             correction_matrix, 
                                                                             epsilon,
                                                                             alpha)


print('total mitigation error:',mitigation_error)

statistical errors: 0.021027423958378727
unphysicality error: 0.004373504085284778
total mitigation error: 0.04926271934109443


Recall once again that since we took a classical noise model from IBM's backend properties, the above error arises
solely due to statistical fluctuations and is underestimated since it does not account for coherent errors. Furthermore,
errors are assumed to be uncorrelated, which underestimates their magnitude.

Note that mitigation error is higher than in case of single-qubit experiments. This is due to the fact that sampling
complexity is higher for 4-dimensional probability distributions. However, we expect that also errors without mitigation
increase:

In [34]:
# We need operational distance between perfect detector and our POVM. We calculate it.
perfect_measurement = povmtools.computational_projectors(d=4) # For two qubits
operational_distance = povmtools.operational_distance_POVMs(ml_povm_estimator, perfect_measurement)
print('errors without mitigation:',operational_distance)

# Now we can check if mitigation can be considered successful
print(f'Mitigation successful: {mitigation_error < operational_distance + epsilon}')

errors without mitigation: 0.13379412421404915
Mitigation successful: True


# References

[0] Filip B. Maciejewski, Zoltán Zimborás, Michał Oszmaniec, *Mitigation of readout noise in near-term quantum devices
by classical post-processing based on detector tomography*, arxiv preprint, https://arxiv.org/abs/1907.08518 (2019)
[1] Z. Hradil, J. Řeháček, J. Fiurášek, and M. Ježek, “3 maximum-likelihood methods in quantum mechanics,” in Quantum
State Estimation, edited by M. Paris and J. Řeháček (Springer Berlin Heidelberg, Berlin, Heidelberg, 2004) pp. 59–112.
